In [1]:
# Last scraped June 10, 2020

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [2]:
ltc_requests = requests.get("http://publicreporting.ltchomes.net/en-ca/Search_Selection.aspx")
ltc = BeautifulSoup(ltc_requests.text)

In [3]:
# ltc_requests.status_code
# ltc_requests.reason
# print(ltc_requests.request.headers)
# ltc_requests.headers

## Webscrape list of LTC home names and links to detailed info

In [4]:
# webscrape list of ltc home names and links
ordered_lists = ltc.find_all("ol")
ltc_list = ordered_lists[1]
home = ltc_list.find_all("a", {"class":"rsLink"})

In [5]:
# create lists of names and links
names = []
links = []
for each in home:
    names.append(each.getText())
    links.append(each.get('href'))

In [6]:
# there are 651 records which is correct
print(len(home), len(names), len(links))

# we see that the names are all unique
print(len(set(names)), len(set(links)))

651 651 651
651 651


In [7]:
# modify links list to full url
full_links = []
for i in range(len(links)):
    full_links.append('http://publicreporting.ltchomes.net/en-ca/' + links[i])

## Webscrape detailed profile info for each LTC home

In [ ]:
# iterate through each of the links and scrape characteristics of ltc homes
addresses = []
cities_postalcodes = []
LHIN = []
licensee = []
management = []
home_type = []
beds = []
short_stay = []
residents_council = []
family_council = []
accreditation = []
info = []
counter = -1
for each in full_links:
    counter += 1
#     print(counter)
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    addresses.append(soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"}).getText())
    cities_postalcodes.append(soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeCity"}).getText())
    profiles = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col1"})
    profile_data = profiles.find_all("div", {"class":"Profilerow_col2"})
    try:    
        LHIN.append(profile_data[0].getText())
        licensee.append(profile_data[3].getText())
        management.append(profile_data[4].getText())
        home_type.append(profile_data[5].getText())
        beds.append(profile_data[6].getText())
        short_stay.append(profile_data[7].getText())
        residents_council.append(profile_data[8].getText())
        family_council.append(profile_data[9].getText())
        accreditation.append(profile_data[10].getText())
        info.append(profile_data[12].getText())
     
    except:
        print(counter) # print out records which resulted in error
        LHIN.append(None)
        licensee.append(None)
        management.append(None)
        home_type.append(None)
        beds.append(None)
        short_stay.append(None)
        residents_council.append(None)
        family_council.append(None)
        accreditation.append(None)
        info.append(None)

### Two homes do not have profile info, tag for removal

In [ ]:
# print out links to LTC homes for which an error was raised during webscraping
print(full_links[324]) # LENNOX AND ADDINGTON COUNTY GENERAL HOSPITAL
print(full_links[508]) # ST. JOSEPH'S MOTHER HOUSE (MARTHA WING)

In [ ]:
# create a df of Ontario LTC homes
df = pd.DataFrame({'name': names, 
                   'address':addresses, 
                   'city_and_postal_code':cities_postalcodes, 
                   'LHIN':LHIN, 
                   'licensee':licensee, 
                   'management':management, 
                   'home_type':home_type, 
                   'beds': beds, 
                   'short_stay':short_stay, 
                   'residents_council':residents_council, 
                   'family_council':family_council, 
                   'accreditation':accreditation, 
                   'additional_info':info})
df.info()
df.head(20)

In [ ]:
# split the 'city and postal code' column into 2 columns 'city' and 'postal code'
df['city'] = df['city_and_postal_code'].str.split(',').str[0]
df['postal_code'] = df['city_and_postal_code'].str.split(',').str[1]

# use regex to extract the number of beds to a different column
df['number_of_beds'] = df['beds'].str.extract(r'(\d+)', expand=False)

df.head(20)
df.nunique()

In [ ]:
# print out LTC homes with duplicate addresses
df[df.duplicated(['address'], keep=False)]

### Manual review of 3 duplicated addresses involving 6 homes and tag for removal as needed
- Cedarwood: no website
- Great Northern: home closed in 2013
- Lakeland Eldcap: no website, seems to be connected to Lakeland LTC
- Lakeland LTC: http://www.lakelandltc.com/, owned and connected to WPSHC which is a hospital
- Harmony: https://www.siennaliving.ca/long-term-care/ontario/harmony-hills-care-community, last inspection Feb 2020
- Fountain: https://www.siennaliving.ca/long-term-care/ontario/fountain-view-care-community, last inspection Oct 2019


## Webscrape inspections info for each LTC home

In [ ]:
# modify links to access inspection data 
full_links_inspection = [each + '&tab=1' for each in full_links]
    

In [ ]:
# webscrape inspection data for each ltc home 
frames = []
counter = -1
for each in full_links_inspection:
    inspection_types = []
    inspection_dates = []
#     counter += 1
#     print(counter)
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    name = soup.find("div", {"class":"HomeName"}).getText()
    inspections = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col3"})
    types = inspections.find_all("div", {"class":"divInspectionTypeDataCol"})
    number_of_types = len(types)
    for each in types:
        inspection_types.append(each.getText())
    dates = inspections.find_all("div", {"class":"divInspectionDateDataCol"})
    number_of_dates = len(dates)
    for each in dates:
        inspection_dates.append(each.getText())
    df_temp = pd.DataFrame({"name": name, 
                            "inspection_types":inspection_types, 
                            "number_of_types":number_of_types, 
                            "inspection_dates":inspection_dates, 
                            "number_of_dates":number_of_dates})
    frames.append(df_temp)
    

### Create a df of raw inspection data

In [ ]:
# create a df of inspection data
df2 = pd.concat(frames)
print(df2.nunique()) # there are only 648 unique LTC home names
df2.info()

# display data for first 2 ltc homes
pd.set_option('display.max_rows', None)
df2.head(10)

In [ ]:
# convert date column into date datatype
df2['inspection_dates'] = pd.to_datetime(df2['inspection_dates'])

In [ ]:
# list 29 different inspection types
df2['inspection_types'].value_counts()

In [ ]:
# Return LTC home names that do not match with original list of names
orig = set(df['name'])
inspec = set(df2['name'])
def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]]
returnNotMatches(orig, inspec)

### Manual review of 3 LTC homes that were not represented in the inspections data and tag for removal
- ST. JOSEPH'S MOTHER HOUSE (MARTHA WING), no inspections or profile information
- NORTHUMBERLAND HILLS HOSPITAL: No inspections, closed 2012
- MARIANHILL - MARGUERITE CENTRE: No inspections, closed 2012


## Enumerate number of inspections for each LTC home

In [ ]:
# for each home identify date of first inspection
min_dates = df2.groupby(['name'])['inspection_dates'].min()
df2['first_inspection_date'] = df2.apply(lambda row: min_dates.loc[row['name']], axis=1)
df2.head()

# create a df with total number of inspections
df_total = df2.groupby(['name']).size().to_frame('total_inspections').reset_index()
df_total.info()

In [ ]:
# create df with date of first inspection
df_first = df2[['name', 'first_inspection_date']]
df_first.drop_duplicates(inplace=True)
df_first.info()
df_first.head()


In [ ]:
# # plot a histogram of total inspections to see how the number of inspections are distributed
# df_total.hist(column='total_inspections')

# # cut total inspections into quartiles and add a column with quartile ranges
# df_total['quartiles_total_range'] = pd.qcut(df_total['total_inspections'], q=4, precision=0)
# print(df_total['quartiles_total_range'].value_counts())

# # add column with quartile rank values
# df_total['quartiles_total_rank'] = pd.qcut(df_total['total_inspections'], q=4, labels = False, precision=0)

# df_total.head()

In [ ]:
# filter df2 and keep only rows with inspections since Jan 2015 inclusive (inspections in the last 5 years)
df_5y = df2[(df2['inspection_dates'] > '2015-01-01')]

# create df with number of inspections in the last 5 years
df_5y = df_5y.groupby(['name']).size().to_frame('5y_inspections')

In [ ]:
# # plot a histogram of 5y inspections to see how the number of inspections are distributed
# df_5y.hist(column='5y_inspections')

# # cut total inspections into quartiles and add a column with quartile ranges
# df_5y['quartiles_5y_range'] = pd.qcut(df_5y['5y_inspections'], q=4, precision=0)
# print(df_5y['quartiles_5y_range'].value_counts())

# # add column with quartile rank values
# df_5y['quartiles_5y_rank'] = pd.qcut(df_5y['5y_inspections'], q=4, labels = False, precision=0)

df_5y.info()
# df_5y.head()


In [ ]:
# filter df2 and keep only rows with inspections since Jan 2018 inclusive (inspections in the last 2 years)
df_2y = df2[(df2['inspection_dates'] > '2018-01-01')]

# create df with number of inspections in the last 2 years
df_2y = df_2y.groupby(['name']).size().to_frame('2y_inspections')

In [ ]:
# # plot a histogram of 2y inspections to see how the number of inspections are distributed
# df_2y.hist(column='2y_inspections')

# # cut total inspections into quartiles and add a column with quartile ranges
# df_2y['quartiles_2y_range'] = pd.qcut(df_2y['2y_inspections'], q=4, precision=0)
# print(df_2y['quartiles_2y_range'].value_counts())

# # add column with quartile rank values
# df_2y['quartiles_2y_rank'] = pd.qcut(df_2y['2y_inspections'], q=4, labels = False, precision=0)

df_2y.info()
# df_2y.head()

### Note that there are fewer homes with inspections in the last 5y/2y meaning that some homes have not been inspected in the last 2-5 years. Some homes may be closed. These have not yet been filtered out. 

## Enumerate the number of inspections containing the words "Compliants", "Critical Incident" and "Order(s)" for each home

In [ ]:
df_complaints = df2[df2["inspection_types"].str.contains('Complaints', regex=False, na=False) ]
df_complaints_total = df_complaints.groupby(['name']).size().to_frame('total_complaints').reset_index()

df_critical = df2[df2["inspection_types"].str.contains('Critical Incident', regex=False, na=False) ]
df_critical_total = df_critical.groupby(['name']).size().to_frame('total_critical').reset_index()

df_withOrders = df2[df2["inspection_types"].str.contains('Order(s)', regex=False, na=False) ]
df_withOrders_total = df_withOrders.groupby(['name']).size().to_frame('total_withOrders').reset_index()


In [ ]:
# Complaints, critical and withOrders in the last 5y
df_complaints = df2[df2["inspection_types"].str.contains('Complaints', regex=False, na=False) ]
df_complaints_5y = df_complaints[(df_complaints['inspection_dates'] > '2015-01-01')]
df_complaints_5y = df_complaints_5y.groupby(['name']).size().to_frame('5y_complaints').reset_index()

df_critical = df2[df2["inspection_types"].str.contains('Critical Incident', regex=False, na=False) ]
df_critical_5y = df_critical[(df_critical['inspection_dates'] > '2015-01-01')]
df_critical_5y = df_critical_5y.groupby(['name']).size().to_frame('5y_critical').reset_index()

df_withOrders = df2[df2["inspection_types"].str.contains('Order(s)', regex=False, na=False) ]
df_withOrders_5y = df_withOrders[(df_withOrders['inspection_dates'] > '2015-01-01')]
df_withOrders_5y = df_withOrders_5y.groupby(['name']).size().to_frame('5y_withOrders').reset_index()

# Complaints, critical and withOrders in the last 2y
df_complaints = df2[df2["inspection_types"].str.contains('Complaints', regex=False, na=False) ]
df_complaints_2y = df_complaints[(df_complaints['inspection_dates'] > '2018-01-01')]
df_complaints_2y = df_complaints_2y.groupby(['name']).size().to_frame('2y_complaints').reset_index()

df_critical = df2[df2["inspection_types"].str.contains('Critical Incident', regex=False, na=False) ]
df_critical_2y = df_critical[(df_critical['inspection_dates'] > '2018-01-01')]
df_critical_2y = df_critical_2y.groupby(['name']).size().to_frame('2y_critical').reset_index()

df_withOrders = df2[df2["inspection_types"].str.contains('Order(s)', regex=False, na=False) ]
df_withOrders_2y = df_withOrders[(df_withOrders['inspection_dates'] > '2018-01-01')]
df_withOrders_2y = df_withOrders_2y.groupby(['name']).size().to_frame('2y_withOrders').reset_index()


In [ ]:
# # Inspect the multiple dataframes
# df_total.info()
# df_5y = df_5y.reset_index(); df_5y.info()
# df_2y = df_2y.reset_index(); df_2y.info()
# df_complaints_total.info() # 9243 complaints
# df_critical_total.info() # 7905 critical 
# df_withOrders_total.info() # 4305 orders
# df_complaints_5y.info() 
# df_critical_5y.info()
# df_withOrders_5y.info()
# df_complaints_2y.info()
# df_critical_2y.info()
# df_withOrders_2y.info()

In [ ]:


# Merge the dataframes on home name
from functools import reduce
list_inspections_dfs = [df_first, df_total, df_5y, df_2y, 
                        df_complaints_total, df_complaints_5y, df_complaints_2y, 
                        df_critical_total, df_critical_5y, df_critical_2y,
                        df_withOrders_total, df_withOrders_5y, df_withOrders_2y]

df_inspections = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'), list_inspections_dfs)
df_inspections.info()
df_inspections.head()

## Merge the inspections data with the profile data

In [ ]:
# Merge the dfs
ltc_scrape = pd.merge(left=df, right=df_inspections, how='left', left_on='name', right_on='name')

# Replace blank values with NaN
ltc_scrape = ltc_scrape.replace(r'^\s*$', np.nan, regex=True)

ltc_scrape.info()
ltc_scrape.head(10)

## Remove inactive homes
- Homes with 'closed' in additional info
- Homes missing all profile information
- Homes with no inspection reports in the last 2 years

In [ ]:
# We see that 20 homes are closed, and one home was merged
ltc_scrape['additional_info'] = ltc_scrape['additional_info'].str.lower()
# ltc_scrape['additional_info'].value_counts()
ltc_scrape.additional_info.str.contains("closed", na=False).value_counts()

In [ ]:
# Drop rows with homes that are closed
ltc_scrape = ltc_scrape[~ltc_scrape.additional_info.str.contains("closed", na=False)].reset_index()

# Review other additional info, there are 5 homes with additional info
ltc_scrape.loc[ltc_scrape['additional_info'].notna()]

## Manual Review of one home that was merged in Jan 2016

- Address to the merged home is just a general PO BOX to the licensee
- Will remove the merged home from the database

In [ ]:
# show rows with information on the 2 homes that were merged
ltc_scrape[ltc_scrape['name'].str.contains('MOUNT HOPE', regex=False, na=False)]

In [ ]:
ltc_scrape.iloc[487]

In [ ]:
# Drop row with the merged home
ltc_scrape = ltc_scrape.drop(ltc_scrape.index[487])

# Check to see it is gone
ltc_scrape[ltc_scrape['name'].str.contains('MOUNT HOPE', regex=False, na=False)]

In [ ]:
# Review homes with missing profile information
ltc_scrape.loc[ltc_scrape['LHIN'].isna()]


### The following homes are missing all profile information:
- LENNOX AND ADDINGTON COUNTY GENERAL HOSPITAL
    - lennox and addington county general hospital: missing all profile information (eg. LHIN, accreditation etc.), https://www.southeasthealthline.ca/displayService.aspx?id=151718, 312, 22 beds convalescent (90 days) and resident long term care at a hospital, last inspection 2019, publically funded, LHIN South East, not in quality database
- ST. JOSEPH'S MOTHER HOUSE (MARTHA WING)
    - no inspections data

In [ ]:
# Drop rows with missing profile information
ltc_scrape = ltc_scrape[ltc_scrape.name != 'LENNOX AND ADDINGTON COUNTY GENERAL HOSPITAL']
ltc_scrape = ltc_scrape[ltc_scrape.name != 'ST. JOSEPH\'S MOTHER HOUSE (MARTHA WING)']
ltc_scrape.loc[ltc_scrape['LHIN'].isna()]

## Review homes without any inspections since January 2018


In [ ]:
ltc_scrape.loc[ltc_scrape['2y_inspections'].isna()]

### The following homes have no inspections data since Jan 2018:
1. LADY ISABELLE NURSING HOME 
    - http://www.ladyisabelle.ca/Contact_Information.html, MISSING from ODHF, missing 2y_inspections, according to web search this home was closed by the ministry in 2014, in april 2020 there are plans to reopen/redevelop a LTC home here, will remove from this analysis
2. MALDEN PARK CONTINUING CARE CENTRE
    - seems to be closed, last inspection 2010, telephone disconnected
3. PEOPLE CARE CENTRE Stratford
    - listed in ODHF but no geodata, https://www.peoplecare.ca/, closed after flood in 2015 per websearch, tel number does not work, missing 5y_inspections

In [ ]:
# Drop rows with no inspections since 2018
ltc_scrape.dropna(subset = ['2y_inspections'], inplace = True)
ltc_scrape.loc[ltc_scrape['2y_inspections'].isna()]

## Export final dataframe with LTC profile and inspections info

In [ ]:
ltc_scrape.info()
ltc_scrape.head()

In [ ]:
# export final df as csv
ltc_scrape.to_csv(r'webscrape_LTC_general_database.csv', index = False)

In [ ]:
# # Test scripts by scraping a single ltc site

# scrape profile data
# soup = requests.get(
# "http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872") 
# soup = BeautifulSoup(soup.text)

# address = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"}).getText()
# city = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeCity"}).getText()
# profiles = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col1"})
# profile_data = profiles.find_all("div", {"class":"Profilerow_col2"})
# profile_data
# LHIN = profile_data[0].getText()
# LHIN

# # scrape inspections data
# soup = requests.get(
# "http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872&tab=1") # add "&tab=1" to url
# soup = BeautifulSoup(soup.text)

# inspections = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col3"})
# inspection_types = inspections.find_all("div", {"class":"divInspectionTypeDataCol"})
# inspection_dates = inspections.find_all("div", {"class":"divInspectionDateDataCol"})

# print(len(inspection_types))
# print(len(inspection_dates))

In [ ]:
# a = []
# for each in inspection_types:
#     a.append(each.getText())
# a

In [ ]:
# df_all.to_csv(r'ltc_covid_odhf_qual_FOR_STATS.csv', index = False)